In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings("ignore")

In [2]:
# matplotlib 한글 나오게 하기

import matplotlib as mpl
import matplotlib.pylab as plt2 

plt2.rcParams["font.family"] = "NanumGothincCoding"
plt2.rcParams["font.size"] = 12
plt2.rcParams["figure.figsize"] = (14, 4) 
mpl.rcParams['axes.unicode_minus'] = False 

In [3]:
raw = pd.read_csv('/home/skinet/hojin/LMS_TEST/(관제)사건사고_인근고객_20200716.csv')

In [4]:
from skinet.utils.athena_tools import *
conn = athena_connection()
df = load_from_athena(conn, '''

SELECT *
  FROM default.monthly_deadline_202004

''') 

...tmp_db name = tmp_20200717_17_26_20_4fc4a374670f0f41c83c4eb195b7aa0c8a8b1058
Start creating table
drop table if exists default.tmp_20200717_17_26_20_4fc4a374670f0f41c83c4eb195b7aa0c8a8b1058
<< table not exist

        create  table         default.tmp_20200717_17_26_20_4fc4a374670f0f41c83c4eb195b7aa0c8a8b1058
        with (format='Parquet', parquet_compression='GZIP')
        as
        

SELECT *
  FROM default.monthly_deadline_202004


.
>> Athena Query Success!!
>> Athena Query EXCUTION COMPLETE....
>> S3 FILE GENERATION SUCCESS!! (File count: 10)
Elapsed Time = 12.42
>> Start downloading table
aws s3 cp s3://adt-sandbox/dsc/tables/29550072-73ae-4d48-ba71-9470b60adaa4 /home/skinet/tmp/tmp_20200717_17_26_20_4fc4a374670f0f41c83c4eb195b7aa0c8a8b1058 --recursive
Elapsed Time = 9.60
>> Loading data
Dropping tmp db
<< Delete s3://adt-sandbox/dsc/tables/29550072-73ae-4d48-ba71-9470b60adaa4
Removing tmp_dir!
Elapsed Time for downloading and loading = 10.01
Total Elapsed Time = 32.04


In [5]:
from skinet.utils.athena_tools import *
conn = athena_connection()
df2 = load_from_athena(conn, '''

SELECT DISTINCT kunnr, telf1, telf2
  FROM sap_raw.kna1

''') 

...tmp_db name = tmp_20200717_17_26_52_e09055785c27b547e61bbc475c5ae316a9c67b29
Start creating table
drop table if exists default.tmp_20200717_17_26_52_e09055785c27b547e61bbc475c5ae316a9c67b29
<< table not exist

        create  table         default.tmp_20200717_17_26_52_e09055785c27b547e61bbc475c5ae316a9c67b29
        with (format='Parquet', parquet_compression='GZIP')
        as
        

SELECT DISTINCT kunnr, telf1,telf2
  FROM sap_raw.kna1


....
>> Athena Query Success!!
>> Athena Query EXCUTION COMPLETE....
>> S3 FILE GENERATION SUCCESS!! (File count: 11)
Elapsed Time = 29.45
>> Start downloading table
aws s3 cp s3://adt-sandbox/dsc/tables/7f038e08-06c1-479b-a2b8-56d4c4de1523 /home/skinet/tmp/tmp_20200717_17_26_52_e09055785c27b547e61bbc475c5ae316a9c67b29 --recursive
Elapsed Time = 11.31
>> Loading data
Dropping tmp db
<< Delete s3://adt-sandbox/dsc/tables/7f038e08-06c1-479b-a2b8-56d4c4de1523
Removing tmp_dir!
Elapsed Time for downloading and loading = 7.04
Total Elapsed Time = 

In [6]:
df['ctnum'] = df['ctnum'].astype('int')

In [195]:
final = pd.merge(raw, df, how = 'left', left_on = '계약번호', right_on = 'ctnum')

In [196]:
final = pd.merge(final, df2, how = 'left', on = 'kunnr')

In [197]:
final.shape

(280, 60)

In [198]:
final['telf1_new'] = final['telf1'].str.replace('-', '')

In [199]:
final['telf2_new'] = final['telf2_y'].str.replace('-', '')

In [200]:
condlist = [final['telf2_new'] != 0, final['telf2_new'] == 0]
choicelist = [final['telf2_new'], final['telf1_new']]

In [201]:
final['telf_num_final'] = np.select(condlist, choicelist)

In [202]:
# final['telf_gukbun'] = final['telf_num_final'].str[0:3]
# final['telf_tf'] = np.where((final['telf_gukbun'] == '010') | (final['telf_gukbun'] == '011') | (final['telf_gukbun'] == '016') |
#                             (final['telf_gukbun'] == '017') | (final['telf_gukbun'] == '018'), True, False)

In [203]:
final['telf_valid_1'] = final['telf_num_final'].str.contains('01{1}[016789]{1}[0-9]{7,8}', regex = True)

In [2]:
def valid_number(x):
      nums = list(str(x)) 
      n = len(nums)
      answer=[]
      i = 3
      
      while i <= n-3:
            if int(nums[i]) - int(nums[i+1]) == int(nums[i+1]) - int(nums[i+2]):
                  answer.append(True)
            else:
                  answer.append(False)
            i += 1
      
      if all(answer) == True:
            return False
      else:
            return True     

In [205]:
final['telf_valid_2'] = final['telf_num_final'].apply(valid_number)

In [208]:
final['telf_valid_final'] = np.where((final['telf_valid_1'] == True) & (final['telf_valid_2'] == True), True, False)

In [210]:
final[final['telf_valid_final'] == False]

,계약번호,rowrank,kcalss,conn_dept,conn_emp,cost_dept,cost_emp,ctnum,cstat,gad_name,...,aenam,con_cnt_per_kunnr,telf1,telf2_y,telf1_new,telf2_new,telf_num_final,telf_valid_1,telf_valid_2,telf_valid_final
9,220107825,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,False
10,220112269,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,False
11,220129569,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,False
71,209502758,1.0,S,200300,K1060191,512200,K1131018,209502758.0,1,*,...,None,1.0,031-875-5601,,0318755601,,,False,False,False
74,211064125,1.0,2,512200,K1110783,512200,K1131018,211064125.0,1,*,...,None,1.0,031-848-8845,1081387601,0318488845,1081387601,1081387601,False,True,False
75,211086280,1.0,S,200300,K1040304,512200,K1110217,211086280.0,1,*,...,None,1.0,02-000-0000,,020000000,,,False,False,False
93,215007656,1.0,S,200300,K1040030,512200,K1110217,215007656.0,1,*,...,None,462.0,0100000000,01000000000,0100000000,01000000000,01000000000,True,False,False
97,215112271,1.0,4,512200,K1050452,512200,K1131018,215112271.0,1,*,...,None,1.0,01000000000,01000000000,01000000000,01000000000,01000000000,True,False,False
104,215176616,1.0,S,200300,K1100004,512200,K1131018,215176616.0,1,*,...,None,1159.0,0,,0,,,False,False,False
115,216106511,1.0,4,512200,K1940305,512200,K1131018,216106511.0,1,*,...,None,1.0,0318405959,0318409595,0318405959,0318409595,0318409595,False,True,False


In [211]:
# LMS 송부 필터링 조건 

cond1 = (final['telf_valid_final'] == True)         # 전화번호 오류제거 
cond2 = (final['gross_type'] != "Y")                # gross제거 1
cond3 = (final['gros_code'] == ' ')                 # gross제거 2
cond4 = (final['adt1_yn'] != "N")                   # 개인정보 수신동의
cond5 = (final['sms_yn'] != "N")                    # sms 수신동의

In [212]:
lms = final[cond1 & cond2 & cond3 & cond4 & cond5]

In [213]:
from sklearn.model_selection import train_test_split
tg, cg = train_test_split(lms, test_size = 0.5, shuffle = True)

In [214]:
print(tg.shape)
print(cg.shape)

(117, 66)
(118, 66)


In [20]:
tg['tg?'] = 1
cg['tg?'] = 0

In [21]:
result = pd.concat([tg, cg])

In [12]:
valid_number('100000')

True